In [18]:
import pandas as pd

In [19]:
# load department labels and hospital department data
df_department_labels = pd.read_csv('../data/in/staging/departments_general_dict.csv', dtype={'department_id': str})
df_department_focus_labels = pd.read_csv('../data/in/staging/departments_focus_dict.csv', dtype={'department_id': str})
df_department_labels_concat = pd.concat([df_department_labels, df_department_focus_labels])
df_hospital_departments = pd.read_csv('../data/in/staging/atlas_departments_0-49.csv')

In [20]:
# rename columns labeled with 'ohne Differenzierung' (no focus specified)
for department_label in list(df_hospital_departments.columns):
    if 'ohne Differenzierung' in str(department_label):
        df_hospital_departments = df_hospital_departments.rename(columns=({department_label: department_label.split('/')[0]}))

# clean columns with '/ ' instead of '/' in the label
for department_label in list(df_hospital_departments.columns):
    if '/ ' in str(department_label):
        df_hospital_departments = df_hospital_departments.rename(columns=({department_label: department_label.replace('/ ', '/')}))

# rename column label 'Neurologie/Schwerpunkt Schlaganfallpatienten (stroke units, Artikel 7 § 1 Abs. 3 GKV-SolG)'
df_hospital_departments = df_hospital_departments.rename(columns=({'Neurologie/Schwerpunkt Schlaganfallpatienten (stroke units, Artikel 7 § 1 Abs. 3 GKV-SolG)': 'Neurologie/Schwerpunkt Schlaganfallpatienten'}))

In [21]:
# create dictionary to translate department labels to department ids
departments_dict = {}
for department_label in df_department_labels_concat['department_label']:
    departments_dict.update({department_label: df_department_labels_concat[df_department_labels_concat['department_label'] == department_label]['department_id'].values[0]})

In [22]:
# apply dictionary to rename columns
df_hospital_departments = df_hospital_departments.rename(columns=departments_dict)

In [23]:
# transform dataframe from wide to long format
df_hospital_departments_long = pd.melt(df_hospital_departments, id_vars=['hospital_id'], var_name='department_id', value_name='treatment_count').dropna()

In [24]:
# save data ready for database import
df_hospital_departments_long.to_csv('../data/db_csv/hospital_departments.csv', index=False, encoding='utf-8')
df_department_labels_concat.to_csv('../data/db_csv/hospital_departments_dict.csv', index=False, encoding='utf-8')